In [ ]:
# Google Colab Project for Calculating vegetation indices for AlUla Project

import ee
try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()


In [ ]:
# The following cell will display a thumbnail of the global elevation model.

# Import the Image function from the IPython.display module.
from IPython.display import Image

# Display a thumbnail of global elevation.
Image(url = dem.updateMask(dem.gt(0))
  .getThumbURL({'min': 0, 'max': 4000, 'dimensions': 512,
                'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}))

In [ ]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Create a folium map object.
my_map = folium.Map(location=[20, 0], zoom_start=3)

# Add the elevation model to the map object.
my_map.add_ee_layer(dem.updateMask(dem.gt(0)), vis_params, 'DEM')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

In [ ]:
# Import the matplotlib.pyplot module.

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


# Fetch a Landsat image.
img = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_034033_20000913')

# Select Red and NIR bands, scale them, and sample 500 points.
samp_fc = img.select(['B3','B4']).divide(10000).sample(scale=30, numPixels=500)

# Arrange the sample as a list of lists.
samp_dict = samp_fc.reduceColumns(ee.Reducer.toList().repeat(2), ['B3', 'B4'])
samp_list = ee.List(samp_dict.get('list'))

# Save server-side ee.List as a client-side Python list.
samp_data = samp_list.getInfo()

# Display a scatter plot of Red-NIR sample pairs using matplotlib.
plt.scatter(samp_data[0], samp_data[1], alpha=0.2)
plt.xlabel('Red', fontsize=12)
plt.ylabel('NIR', fontsize=12)
plt.show()


In [ ]:


# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap', 'descartes'])

import ee
import geemap

In [ ]:
#This is only required if running in colab notebook to install the libraries
#If running Python code elsewhere, need to make sure below libraries are installed
! pip install geopandas
! pip install descartes
! pip install rasterio
! pip install rasterstats

In [ ]:
# Connect to your Google Drive in order to save the images we'll be collecting later

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from pathlib import Path
#from jmd_imagescraper.core import *

project = "AOI_Project"
path = Path(project)

folder_path = f'Data-image-classifier/data/{project}'
file_name = f'{project}.tgz'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import json
from sklearn.model_selection import train_test_split
import geopandas as gpd
import descartes
import rasterio
from rasterio.mask import mask
from rasterio.features import geometry_mask
from shapely.geometry import mapping
from rasterstats import zonal_stats
import datetime

In [ ]:

from urllib.request import urlopen
import ee
import geemap
import os
from datetime import date, datetime, timedelta

import requests
import json
import pprint


import numpy as np

#import descartes
import rasterio


S2_MSI_Level1C = ee.ImageCollection("COPERNICUS/S2")
SAU_AOI = ee.FeatureCollection("users/Frednumbisi/SAU_AOI")

Al_AOI = ee.FeatureCollection("projects/ee-frednumbisi/assets/AOIProject")


Al_ROI = Al_AOI.geometry()


# Create and interactive map
Map = geemap.Map(center=(40, -100), zoom=4)

# center the map on an Earth Engine object:
# Map.centerObject(ee_object=xy, zoom=13)
Map.centerObject(ee_object=Al_AOI, zoom=13); # Get centre map to AlUla


In [ ]:
# set your root directory to the shared drive folder
# root_dir = '/content/drive/Shared drives/servir-sat-ml/data/'
root_dir = '/content/drive/My Drive/'

# go to root directory
%cd $root_dir

In [ ]:
Al_ROI.plot(figsize=(10, 10));

In [ ]:

from urllib.request import urlopen
import ee
import geemap
import os
from datetime import date, datetime, timedelta

import requests
import json
import pprint


import numpy as np
import geopandas as gpd

#import descartes
import rasterio


S2_MSI_Level1C = ee.ImageCollection("COPERNICUS/S2")

# Create function that adds a band representing the image timestamp.
def addTime(image):
  return image.addBands(image.metadata('system:time_start').divide(1000 * 60 * 60 * 24 * 365))


# Load a Sentinel-2 collection
s2Collection = S2_MSI_Level1C
NIR = "B8"  # Name variable for NIR band (B8 for Sentinel2, or B5 for Landsat)

# Filter data collection over time period and bands (Red, Green, Blue, and NIR)
# On 2nd May 2023, I updated image time series collection interval to Sept 30th 2023 on
# and map the time band function over it.
s2Filtered = ee.ImageCollection("COPERNICUS/S2").filterDate('2021-01-01', '2023-09-30') \
                         .filterBounds(Al_AOI) \
                         .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 1) \
                         .map(addTime)\
                         .select(['B4', 'B3', 'B2', NIR])

# Create visualisation parameter
rgb_vis = {
  'min': 0,
  'max': 2600,
  'gamma': 1.4,
}

# Create variable to compute and add NDVI image
def calNDVI(img):
  ndvi = img.normalizedDifference([NIR, 'B4']).rename('NDVI')
  return img.addBands(ndvi)

def addNDVI(image):
  ndvi = image.normalizedDifference([NIR, 'B4']).rename('NDVI')
  return image.addBands(ndvi)

def get_days(date):
  m = date.millis()
  return m.divide(1000).divide(3600).toInt()

# Create variable to add a 'date' band: number of days since epoch
def addDate(img):
  d = ee.Date(img.date())
  days = get_days(d)
  days_img = ee.Image.constant(days).rename('date').toInt32()
  return img.addBands(days_img)


#-----------------------------------------------------------------------------
# // compute the infrared percentage vegetation index (ndvi)
def compute_ndvi(image):
    nir = image.select('B8').divide(10000)
    red = image.select('B4').divide(10000)
    ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
    invndvi = ee.Image.constant(0).subtract(ndvi).rename('invndvi')
    return image.addBands(ndvi).addBands(invndvi)

plusNDVI = s2Filtered.map(compute_ndvi)


# / Get the 25th and 75th percentile NDVI value for each pixe in image collection
def filter_ndvi75(image):
  ndvi_75 = image.reduce(ee.Reducer.percentile([75]))
  ndvi_25 = image.reduce(ee.Reducer.percentile([25]))
  lte75_mask = image.select('NDVI').lte(ndvi_75)
  image = image.updateMask(lte75_mask.select('NDVI'))
  image = image.updateMask(image.select('NDVI').gte(ndvi_25))
  return image

filtered_ndvi75 = plusNDVI.map(filter_ndvi75)

BestNDVI25_75perc = filtered_ndvi75.qualityMosaic('NDVI').clip(Al_AOI)

#BestNDVI25_75percB = filtered_ndvi75.qualityMosaic('NDVI').clip(Al_AOI).astype(rasterio.uint8)

# ndviParams = {min: -1, max: 1, palette: ['blue', 'white', 'green']};
# Map.addLayer(ndvi, ndviParams, 'NDVI image');
# #print(BestNDVI25_95perc)


meanNDVI25_75perc = filtered_ndvi75.mean()
stdNDVI25_75perc = filtered_ndvi75.reduce(ee.Reducer.stdDev())



# Create and interactive map

Map = geemap.Map(center=(40, -100), zoom=4)



# center the map on an Earth Engine object:
# Map.centerObject(ee_object=xy, zoom=13)
Map.centerObject(ee_object=Al_AOI, zoom=13); # Get centre map to AlUla




# COMPUTE VEGETATION INDICES
# ------------------------------------------------------------
# NDVI
# // compute the normalised difference vegetation index (ndvi)
def compute_ndvi(image):
    nir = image.select('B8')
    red = image.select('B4')
    ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
    invndvi = ee.Image.constant(0).subtract(ndvi).rename('invndvi')
    return image.addBands(ndvi).addBands(invndvi)

plusNDVI = s2Filtered.map(compute_ndvi)


# / Get the 25th and 75th percentile NDVI value for each pixe in image collection
def filter_ndvi75(image):
  ndvi_75 = image.reduce(ee.Reducer.percentile([75]))
  ndvi_25 = image.reduce(ee.Reducer.percentile([25]))
  lte75_mask = image.select('NDVI').lte(ndvi_75)
  image = image.updateMask(lte75_mask.select('NDVI'))
  image = image.updateMask(image.select('NDVI').gte(ndvi_25))
  return image


filtered_ndvi75 = plusNDVI.map(filter_ndvi75)
#print(filtered_ndvi95)

BestNDVI25_75perc = filtered_ndvi75.qualityMosaic('NDVI').clip(Al_AOIROI)
#print(BestNDVI25_95perc)

# meanNDVI25_75perc = filtered_ndvi75.mean()
meanNDVI25_75perc = filtered_ndvi75.reduce(ee.Reducer.mean())
stdNDVI25_75perc = filtered_ndvi75.reduce(ee.Reducer.stdDev())
minNDVI25_75perc = filtered_ndvi75.reduce(ee.Reducer.min())
maxNDVI25_75perc = filtered_ndvi75.reduce(ee.Reducer.max())
medNDVI25_75perc = filtered_ndvi75.reduce(ee.Reducer.median())



# ---------------------------------------------------------------------------
# SAVI

# Function to compute SAVI
def compute_savi(image):
    # compute the soil-adjusted vegetation index (savi)
    savi = image.expression('1.5 * (NIR - RED) / (NIR + RED + 0.5)', {
        'NIR': image.select('B8').divide(10000),
        'RED': image.select('B4').divide(10000)}).rename('SAVI')
    # return image.addBands(gsavi)
    # get inverted gsavi
    invsavi = ee.Image.constant(0).subtract(savi).rename('invsavi')
    return image.addBands(savi).addBands(invsavi)

plusSAVI = s2Filtered.map(compute_savi)


def filter_savi75(image):
    # Get the 75th percentile NDVI value for image
    savi_75 = image.reduce(ee.Reducer.percentile([75]))
    # Get the 25th percentile NDVI value for image
    savi_25 = image.reduce(ee.Reducer.percentile([25]))
    # Create a mask selecting all values less than the 75th percentile
    lte75_mask = image.select('SAVI').lte(savi_75)
    # Update the image removing all values > the 75th percentile
    image = image.updateMask(lte75_mask.select('SAVI'))
    # Update the image removing all values < the 25th percentile
    image = image.updateMask(image.select('SAVI').gte(savi_25))
    return image



# filter image collection by NDVI 95th percentile
filtered_savi75 = plusSAVI.map(filter_savi75)

print(filtered_savi75)

# create greenest pixel quality mosaic
BestSAVI25_75perc = filtered_savi75.qualityMosaic('SAVI').clip(Al_AOIROI)


# leastGSAVI25_95perc = filtered_gsavi95.qualityMosaic('invgsavi')
# meanSAVI25_75perc = filtered_savi75.mean()
meanSAVI25_75perc = filtered_savi75.reduce(ee.Reducer.mean())
stdSAVI25_75perc = filtered_savi75.reduce(ee.Reducer.stdDev());
minSAVI25_75perc = filtered_savi75.reduce(ee.Reducer.min());
maxSAVI25_75perc = filtered_savi75.reduce(ee.Reducer.max());
medSAVI25_75perc = filtered_savi75.reduce(ee.Reducer.median());




# Define the function to compute MSAVI2
def compute_msavi2(image):
    msavi = image.expression('0.5 * ((2 * NIR + 1) - (sqrt(pow((2 * NIR + 1), 2) - 8 * (NIR - RED)) ) )', {
        'NIR': image.select('B8').divide(10000),
        'RED': image.select('B4').divide(10000)}).rename('MSAVI')
    invmsavi = ee.Image.constant(0).subtract(msavi).rename('invmsavi')
    return image.addBands(msavi).addBands(invmsavi).float()

plusMSAVI2 = s2Filtered.map(compute_msavi2)


def filter_msavi75(image):
    msavi_75 = image.reduce(ee.Reducer.percentile([75]))
    msavi_25 = image.reduce(ee.Reducer.percentile([25]))
    lte75_mask = image.select('MSAVI').lte(msavi_75)
    image = image.updateMask(lte75_mask.select('MSAVI'))
    image = image.updateMask(image.select('MSAVI').gte(msavi_25))
    return image

filtered_msavi2575 = plusMSAVI2.map(filter_msavi75)

# meanMSAVI25_75perc = filtered_msavi2575.mean()
meanMSAVI25_75perc = filtered_msavi2575.reduce(ee.Reducer.mean())
stdMSAVI25_75perc = filtered_msavi2575.reduce(ee.Reducer.stdDev())
minMSAVI25_75perc = filtered_msavi2575.reduce(ee.Reducer.min())
maxMSAVI25_75perc = filtered_msavi2575.reduce(ee.Reducer.max())
medMSAVI25_75perc = filtered_msavi2575.reduce(ee.Reducer.median())


#-------------------------------------------------------------------------------
# GSAVI


def compute_gsavi(image):
    # compute the soil-adjusted vegetation index (savi)
    gsavi = image.expression('1.5 * (NIR - GREEN) / (NIR + GREEN + 0.5)', {
        'NIR': image.select('B8').divide(10000),
        'GREEN': image.select('B3').divide(10000)}).rename('GSAVI')
    # return image.addBands(gsavi)
    # get inverted gsavi
    invgsavi = ee.Image.constant(0).subtract(gsavi).rename('invgsavi')
    return image.addBands(gsavi).addBands(invgsavi)


plusGSAVI = s2Filtered.map(compute_gsavi)



def filter_gsavi75(image):
    gsavi_75 = image.reduce(ee.Reducer.percentile([75]))
    gsavi_25 = image.reduce(ee.Reducer.percentile([25]))
    lte75_mask = image.select('GSAVI').lte(gsavi_75)
    image = image.updateMask(lte75_mask.select('GSAVI'))
    image = image.updateMask(image.select('GSAVI').gte(gsavi_25))
    return image

filtered_gsavi2575 = plusGSAVI.map(filter_gsavi75)

# meanGSAVI25_75perc = filtered_gsavi2575.mean()
meanGSAVI25_75perc = filtered_gsavi2575.reduce(ee.Reducer.mean())
stdGSAVI25_75perc = filtered_gsavi2575.reduce(ee.Reducer.stdDev())
minGSAVI25_75perc = filtered_gsavi2575.reduce(ee.Reducer.min())
maxGSAVI25_75perc = filtered_gsavi2575.reduce(ee.Reducer.max())
medGSAVI25_75perc = filtered_gsavi2575.reduce(ee.Reducer.median())


# ------------------------------------------------------------------------------
# IPVI

def compute_ipvi(image):
  # compute the green-red vegetation index (grvi)
    ipvi = image.expression('NIR/(NIR + RED)', {
      'NIR': image.select('B8').divide(10000),
      'RED': image.select('B4').divide(10000)}).rename('IPVI');
    # get inverted grvi
    invipvi = ee.Image.constant(0).subtract(ipvi).rename('invipvi')
    return image.addBands(ipvi).addBands(invipvi)


plusIPVI = s2Filtered.map(compute_ipvi)




def IPVIfilter(image):
    ipvi_75 = image.reduce(ee.Reducer.percentile([75]))
    ipvi_25 = image.reduce(ee.Reducer.percentile([25]))
    lte75_mask = image.select('IPVI').lte(ipvi_75)
    image = image.updateMask(lte75_mask.select('IPVI'))
    image = image.updateMask(lte75_mask.select('IPVI').gte(ipvi_25))
    return image

filtered_ipvi2575 = plusIPVI.map(IPVIfilter)

# meanIPVI25_75perc = filtered_ipvi2575.mean()
meanIPVI25_75perc = filtered_ipvi2575.reduce(ee.Reducer.mean())
stdIPVI25_75perc = filtered_ipvi2575.reduce(ee.Reducer.stdDev())
minIPVI25_75perc = filtered_ipvi2575.reduce(ee.Reducer.min())
maxIPVI25_75perc = filtered_ipvi2575.reduce(ee.Reducer.max())
medIPVI25_75perc = filtered_ipvi2575.reduce(ee.Reducer.median())


#----------------------------------------------------
# SARVI


# Define the function to compute SARVI
def compute_sarvi(image):
    sarvi = image.expression('(NIR - (RED - 1 * (BLUE - RED))) / (NIR + (RED - 1 * (BLUE - RED)))', {
        'NIR': image.select('B8').divide(10000),
        'BLUE': image.select('B2').divide(10000),
        'RED': image.select('B4').divide(10000)}).rename('SARVI')
    invsarvi = ee.Image.constant(0).subtract(sarvi).rename('invsarvi')
    return image.addBands(sarvi).addBands(invsarvi)

def filter_sarvi75(image):
    sarvi_75 = image.reduce(ee.Reducer.percentile([75]))
    sarvi_25 = image.reduce(ee.Reducer.percentile([25]))
    lte75_mask = image.select('SARVI').lte(sarvi_75)
    image = image.updateMask(lte75_mask.select('SARVI'))
    image = image.updateMask(lte75_mask.select('SARVI').gte(sarvi_25))
    return image

# compute SARVI for each image in the image collection using the .map()
plusSARVI = s2Filtered.map(compute_sarvi)

# Filter each image to the middle 50 percentile (IQR) for the values in each pixel
filtered_sarvi2575 = plusSARVI.map(filter_sarvi75)


# Reduce the Image Collection to Mean and Standard Deviation for each band in the filtered Image
meanSARVI25_75perc = filtered_sarvi2575.reduce(ee.Reducer.mean())
stdSARVI25_75perc = filtered_sarvi2575.reduce(ee.Reducer.stdDev())
minSARVI25_75perc = filtered_sarvi2575.reduce(ee.Reducer.min())
maxSARVI25_75perc = filtered_sarvi2575.reduce(ee.Reducer.max())
medSARVI25_75perc = filtered_sarvi2575.reduce(ee.Reducer.median())


#-----------------------------------------------------
# GRVI
# Define the function to compute GRVI

def compute_grvi(image):
  # compute the green-red vegetation index (grvi)
    grvi = image.expression('(GREEN - RED) / (GREEN + RED)', {
      'GREEN': image.select('B3').divide(10000),
      'RED': image.select('B4').divide(10000)}).rename('GRVI');
    # get inverted grvi
    invgrvi = ee.Image.constant(0).subtract(grvi).rename('invgrvi')

    return image.addBands(grvi).addBands(invgrvi)


def GRVIfilter(image):
    grvi_75 = image.reduce(ee.Reducer.percentile([75]))
    grvi_25 = image.reduce(ee.Reducer.percentile([25]))
    lte75_mask = image.select('GRVI').lte(grvi_75)
    image = image.updateMask(lte75_mask.select('GRVI'))
    image = image.updateMask(lte75_mask.select('GRVI').gte(grvi_25))
    return image

plusGRVI = s2Filtered.map(compute_grvi)
filtered_grvi2575 = plusGRVI.map(GRVIfilter)

meanGRVI25_75perc = filtered_grvi2575.mean()


# Use the non-default mean reducer for image collection
# This will append the name of reducer to the additional bands
meanGRVI25_75perc = filtered_grvi2575.reduce(ee.Reducer.mean())

# Compute standard deviation (SD) as texture of the vegetation index (GRVI).
stdGRVI25_75perc = filtered_grvi2575.reduce(ee.Reducer.stdDev())

# Reduce the coposites to single median of the vegetation index (GRVI).
#medGRVI25_75perc = filtered_grvi2575.reduce(ee.Reducer.median())
minGRVI25_75perc = filtered_grvi2575.reduce(ee.Reducer.min())
maxGRVI25_75perc = filtered_grvi2575.reduce(ee.Reducer.max())
medGRVI25_75perc = filtered_grvi2575.reduce(ee.Reducer.median())



# Reduce GRV IQR to max value and clip image to area of interest
BestGRVI25_75perc = filtered_grvi2575.qualityMosaic('GRVI').clip(Al_ROI)

from pprint import pprint

# Create combined reducer and map to image collection
CombiReducers = ee.Reducer.mean().combine(
  reducer2=ee.Reducer.stdDev(),
  sharedInputs=True
)

# Use the combined reducer to get the mean and SD bands for each band image.
GRVIstats = filtered_grvi2575.reduce(
  reducer=CombiReducers)

# Display the dictionary of band means and SDs.
pprint(GRVIstats.getInfo())


#-------------------------------------------------------------------------
# CREATE A COMPOSITE IMAGES FOR MEAN AND STD OF VEGETATION INDICES

def stackCollection(collection):
  # Create an initial image.
  first = ee.Image(collection.first()).select([])

  # Write a function that appends a band to an image.
  def appendBands(image, previous):
    return ee.Image(previous).addBands(image)
  return ee.Image(collection.iterate(appendBands, first))



# Cast each image to float32 bit
compositeMeanVIndices = meanNDVI25_75perc.select(['NDVI_mean'])\
  .addBands(meanSAVI25_75perc.select(['SAVI_mean']))\
  .addBands(meanGSAVI25_75perc.select(['GSAVI_mean']))\
  .addBands(meanMSAVI25_75perc.select(['MSAVI_mean']))\
  .addBands(meanIPVI25_75perc.select(['IPVI_mean']))\
  .addBands(meanSARVI25_75perc.select(['SARVI_mean']))\
  .addBands(meanGRVI25_75perc.select(['GRVI_mean']))




#-------------------------------------------------------------------------------
# Function to mask all values outside the geometry
def maskOutside(image, geometry):
    mask = ee.Image.constant(1).clip(geometry).mask() # add .not() to mask inside
    return image.updateMask(mask)

#-------------------------------------------------------------------------------

# Create a composite from Standard deviation of different vegetation indices.
compositeStDevVIndices = stdNDVI25_75perc.select(['NDVI_stdDev'])\
  .addBands(stdSAVI25_75perc.select(['SAVI_stdDev']))\
  .addBands(stdGSAVI25_75perc.select(['GSAVI_stdDev']))\
  .addBands(stdMSAVI25_75perc.select(['MSAVI_stdDev']))\
  .addBands(stdIPVI25_75perc.select(['IPVI_stdDev']))\
  .addBands(stdSARVI25_75perc.select(['SARVI_stdDev']))\
  .addBands(stdGRVI25_75perc.select(['GRVI_stdDev']))


# Create Composite of MaxIQR of VI
compositeMaxVIndices = maxNDVI25_75perc.select(['NDVI_max'])\
  .addBands(maxSAVI25_75perc.select(['SAVI_max']))\
  .addBands(maxGSAVI25_75perc.select(['GSAVI_max']))\
  .addBands(maxMSAVI25_75perc.select(['MSAVI_max']))\
  .addBands(maxIPVI25_75perc.select(['IPVI_max']))\
  .addBands(maxSARVI25_75perc.select(['SARVI_max']))\
  .addBands(maxGRVI25_75perc.select(['GRVI_max']))


# Create Composite of MinIQR of VI
compositeMinVIndices = minNDVI25_75perc.select(['NDVI_min'])\
  .addBands(minSAVI25_75perc.select(['SAVI_min']))\
  .addBands(minGSAVI25_75perc.select(['GSAVI_min']))\
  .addBands(minMSAVI25_75perc.select(['MSAVI_min']))\
  .addBands(minIPVI25_75perc.select(['IPVI_min']))\
  .addBands(minSARVI25_75perc.select(['SARVI_min']))\
  .addBands(minGRVI25_75perc.select(['GRVI_min']))


# Create Composite of MinIQR of All Spectral Bands for Computed VIs
compositeAllVIspectralBands = meanNDVI25_75perc.select(['B4_mean', 'B3_mean','B2_mean','B8_mean'])\
  .addBands(meanSAVI25_75perc.select(['B4_mean', 'B3_mean','B2_mean','B8_mean']))\
  .addBands(meanGSAVI25_75perc.select(['B4_mean', 'B3_mean','B2_mean','B8_mean']))\
  .addBands(meanMSAVI25_75perc.select(['B4_mean', 'B3_mean','B2_mean','B8_mean']))\
  .addBands(meanIPVI25_75perc.select(['B4_mean', 'B3_mean','B2_mean','B8_mean']))\
  .addBands(meanSARVI25_75perc.select(['B4_mean', 'B3_mean','B2_mean','B8_mean']))\
  .addBands(meanGRVI25_75perc.select(['B4_mean', 'B3_mean','B2_mean','B8_mean']))\
  .addBands(minNDVI25_75perc.select(['B4_min', 'B3_min','B2_min','B8_min']))\
  .addBands(minSAVI25_75perc.select(['B4_min', 'B3_min','B2_min','B8_min']))\
  .addBands(minGSAVI25_75perc.select(['B4_min', 'B3_min','B2_min','B8_min']))\
  .addBands(minMSAVI25_75perc.select(['B4_min', 'B3_min','B2_min','B8_min']))\
  .addBands(minIPVI25_75perc.select(['B4_min', 'B3_min','B2_min','B8_min']))\
  .addBands(minSARVI25_75perc.select(['B4_min', 'B3_min','B2_min','B8_min']))\
  .addBands(minGRVI25_75perc.select(['B4_min', 'B3_min','B2_min','B8_min']))\
  .addBands(maxNDVI25_75perc.select(['B4_max', 'B3_max','B2_max','B8_max']))\
  .addBands(maxSAVI25_75perc.select(['B4_max', 'B3_max','B2_max','B8_max']))\
  .addBands(maxGSAVI25_75perc.select(['B4_max', 'B3_max','B2_max','B8_max']))\
  .addBands(maxMSAVI25_75perc.select(['B4_max', 'B3_max','B2_max','B8_max']))\
  .addBands(maxIPVI25_75perc.select(['B4_max', 'B3_max','B2_max','B8_max']))\
  .addBands(maxSARVI25_75perc.select(['B4_max', 'B3_max','B2_max','B8_max']))\
  .addBands(maxGRVI25_75perc.select(['B4_max', 'B3_max','B2_max','B8_max']))\
  .addBands(stdNDVI25_75perc.select(['B4_stdDev', 'B3_stdDev','B2_stdDev','B8_stdDev']))\
  .addBands(stdSAVI25_75perc.select(['B4_stdDev', 'B3_stdDev','B2_stdDev','B8_stdDev']))\
  .addBands(stdGSAVI25_75perc.select(['B4_stdDev', 'B3_stdDev','B2_stdDev','B8_stdDev']))\
  .addBands(stdMSAVI25_75perc.select(['B4_stdDev', 'B3_stdDev','B2_stdDev','B8_stdDev']))\
  .addBands(stdIPVI25_75perc.select(['B4_stdDev', 'B3_stdDev','B2_stdDev','B8_stdDev']))\
  .addBands(stdSARVI25_75perc.select(['B4_stdDev', 'B3_stdDev','B2_stdDev','B8_stdDev']))\
  .addBands(stdGRVI25_75perc.select(['B4_stdDev', 'B3_stdDev','B2_stdDev','B8_stdDev']))



# Create Composite of MinIQR of All Spectral Bands for Computed VIs
compositeVIspectralBands = meanNDVI25_75perc.select(['B4_mean', 'B3_mean','B2_mean','B8_mean'])\
  .addBands(minNDVI25_75perc.select(['B4_min', 'B3_min','B2_min','B8_min']))\
  .addBands(maxNDVI25_75perc.select(['B4_max', 'B3_max','B2_max','B8_max']))\
  .addBands(stdNDVI25_75perc.select(['B4_stdDev', 'B3_stdDev','B2_stdDev','B8_stdDev']))



# # Earth Engine layers to Map
Map.addLayer(meanNDVI25_75perc, rgb_vis, "Mean VI Composite 25th to 75th Perc.")
Map.addLayer(meanMSAVI25_75perc, rgb_vis, "Mean MSAVI2 Composite 25th to 75th Perc.")
Map.addLayer(meanIPVI25_75perc, rgb_vis, "Mean IPVI Composite 25th to 75th Perc.")

Map.addLayer(stdGRVI25_75perc, rgb_vis, "StDev GRVI Composite 25th to 75th Perc.")
Map.addLayer(stdSARVI25_75perc, rgb_vis, "StDev SARVI Composite 25th to 75th Perc.")
Map.addLayer(stdNDVI25_75perc, rgb_vis, "StDev NDVI Composite 25th to 75th Perc.")

Map.addLayer(minGRVI25_75perc, rgb_vis, "Min GRVI IQR")

Map.addLayer(compositeMeanVIndices, rgb_vis, "Mean VI Composite 25th to 75th Perc.")

In [ ]:
!pip install earthpy
!pip install geopandas
!pip install rasterio

!apt-get install -qq libgdal-dev libproj-dev
!pip install cartopy


In [ ]:
import cartopy
import pandas as pd
import geopandas as gpd
import folium
import os, shutil
from glob import glob
import numpy as np
import rasterio as rio
from rasterio.transform import from_origin
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from rasterio.plot import show, plotting_extent
from rasterio.merge import merge
from rasterio.warp import calculate_default_transform, reproject, Resampling
import earthpy.spatial as es
import earthpy.plot as ep



In [ ]:
!pip install wxee

In [ ]:
#%matplotlib inline

from sklearn.decomposition import PCA
import geopandas as gpd
import numpy as np
import pandas as pd
import cv2 as cv
from google.colab.patches import cv2_imshow
from osgeo import gdal

import json
import wxee
import ee
import geemap
#import sklearn_flatten, sklearn_unflatten


In [ ]:
# Import the necessary libraries
from PIL import Image
from numpy import asarray

In [ ]:
# To perform a PCA, data is first transformed into a numpy array that can be used by sklearn
# compositeMeanVIndices
# compositeStDevVIndices
# compositeMaxVIndices
# compositeMinVIndices
# compositeVIspectralBands # 16 spectral Bands

CompostMeanVI_Coll = ee.ImageCollection(compositeMeanVIndices)
CompostMeanVI_Coll.getInfo()


In [ ]:
compositeMeanVI_coll_fin = maskOutside(CompostMeanVI_Coll, Al_AOI)

In [ ]:
compositeMeanVI_coll = ee.ImageCollection(compositeMeanVIndices)

compositeMeanVI_coll.getInfo()

In [ ]:
# Export the image, specifying scale and region.
# Export the Standardised IQR Images

task = ee.batch.Export.image.toDrive(
    image = compositeMeanVIndices,
    region = Al_AOI.geometry().bounds(), # Or use custom ee.Geometry.Rectangle([minlon, minlat, maxlon, maxlat])
    description = '2023AOI_MeanVI_Composit',
    scale = 10,
    folder = "AOI_VIIQR_Composits",
    crs = 'EPSG:4326',
    #crs ='EPSG:3857', # UTM CRS for Saudi
    #crsTransform = proj["transform"],
    maxPixels = 1e13,
    fileFormat = "GeoTIFF",
    formatOptions = {
    'cloudOptimized': True
  }
    )

task.start()

import time
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

In [ ]:
# Export the image, specifying scale and region.
task = ee.batch.Export.image.toDrive(**{
    'image': l8_image,
    'description': 'Landsat8_2020_Nepal',
    'folder': output_drive_folder,
    'fileNamePrefix': raster_name,
    'scale': 30,
    'region': gee_aoi,
    'fileFormat': 'GeoTIFF',
    'formatOptions': {
      'cloudOptimized': 'true'
    },
})


In [ ]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
filename = os.path.join(out_dir, 'MeanVI.tif')

AOI = Al_AOI.geometry()

# Convert the reduced mean images to float23
NDVImean = ee.Image(meanNDVI25_75perc.select(['NDVI_mean']).float())
SAVImean = ee.Image(meanSAVI25_75perc.select(['SAVI_mean']).float())
GSAVImean = ee.Image(meanGSAVI25_75perc.select(['GSAVI_mean']).float())
MSAVImean = ee.Image(meanMSAVI25_75perc.select(['MSAVI_mean']).float())
IPVImean = ee.Image(meanIPVI25_75perc.select(['IPVI_mean']).float())
SARVImean = ee.Image(meanSARVI25_75perc.select(['SARVI_mean']).float())
GRVImean = ee.Image(meanGRVI25_75perc.select(['GRVI_mean']).float())


In [ ]:
# Apply the maskOutside() function to clip image and Replace all values outside region or No Data with -9999

#image = maskOutside(compositeMeanVIndices, Al_AOI).unmask(-9999)

image_VImean = maskOutside(compositeMeanVIndices, Al_AOI)

image_NDVImean = maskOutside(NDVImean, Al_AOI)
image_SAVImean = maskOutside(SAVImean, Al_AOI)
image_GSAVImean = maskOutside(GSAVImean, Al_AOI)
image_MSAVImean = maskOutside(MSAVImean, Al_AOI)
image_IPVImean = maskOutside(IPVImean, Al_AOI)
image_SARVImean = maskOutside(SARVImean, Al_AOI)
image_GRVImean = maskOutside(GRVImean, Al_AOI)


#image_std = maskOutside(compositeStDevVIndices, Al_AOI).unmask(-9999)
image_std = maskOutside(compositeStDevVIndices, Al_AOI)

In [ ]:
# Convert each image of standarDeviation of IQR to float32: To be used for texture-based predictors
# Apply the maskOutside() function to clip image and Replace all values outside region or No Data with -9999

image_NDVIstD = maskOutside(ee.Image(stdNDVI25_75perc.select(['NDVI_stdDev']).float()), Al_AOI)
image_SAVIstD = maskOutside(ee.Image(stdSAVI25_75perc.select(['SAVI_stdDev']).float()), Al_AOI)
image_GSAVIstD = maskOutside(ee.Image(stdGSAVI25_75perc.select(['GSAVI_stdDev']).float()), Al_AOI)
image_MSAVIstD = maskOutside(ee.Image(stdMSAVI25_75perc.select(['MSAVI_stdDev']).float()), Al_AOI)
image_IPVIstD = maskOutside(ee.Image(stdIPVI25_75perc.select(['IPVI_stdDev']).float()), Al_AOI)
image_SARVIstD = maskOutside(ee.Image(stdSARVI25_75perc.select(['SARVI_stdDev']).float()), Al_AOI)
image_GRVIstD = maskOutside(ee.Image(stdGRVI25_75perc.select(['GRVI_stdDev']).float()), Al_AOI)

In [ ]:
# Convert each image of minimum IQR to float32:

image_NDVImin = maskOutside(ee.Image(minNDVI25_75perc.select(['NDVI_min']).float()), Al_AOI)
image_SAVImin = maskOutside(ee.Image(minSAVI25_75perc.select(['SAVI_min']).float()), Al_AOI)
image_GSAVImin = maskOutside(ee.Image(minGSAVI25_75perc.select(['GSAVI_min']).float()), Al_AOI)
image_MSAVImin = maskOutside(ee.Image(minMSAVI25_75perc.select(['MSAVI_min']).float()), Al_AOI)
image_IPVImin = maskOutside(ee.Image(minIPVI25_75perc.select(['IPVI_min']).float()), Al_AOI)
image_SARVImin = maskOutside(ee.Image(minSARVI25_75perc.select(['SARVI_min']).float()), Al_AOI)
image_GRVImin = maskOutside(ee.Image(minGRVI25_75perc.select(['GRVI_min']).float()), Al_AOI)



In [ ]:
# Conver each image of median IQR to float32

image_NDVImed = maskOutside(ee.Image(medNDVI25_75perc.select(['NDVI_median']).float()), Al_AOI)
image_SAVImed = maskOutside(ee.Image(medSAVI25_75perc.select(['SAVI_median']).float()), Al_AOI)
image_GSAVImed = maskOutside(ee.Image(medGSAVI25_75perc.select(['GSAVI_median']).float()), Al_AOI)
image_MSAVImed = maskOutside(ee.Image(medMSAVI25_75perc.select(['MSAVI_median']).float()), Al_AOI)
image_IPVImed = maskOutside(ee.Image(medIPVI25_75perc.select(['IPVI_median']).float()), Al_AOI)
image_SARVImed = maskOutside(ee.Image(medSARVI25_75perc.select(['SARVI_median']).float()), Al_AOI)
image_GRVImed = maskOutside(ee.Image(medGRVI25_75perc.select(['GRVI_median']).float()), Al_AOI)


In [ ]:
# Convert the reduced image of maximum IQR to float32

image_NDVImax = maskOutside(ee.Image(maxNDVI25_75perc.select(['NDVI_max']).float()), Al_AOI)
image_SAVImax = maskOutside(ee.Image(maxSAVI25_75perc.select(['SAVI_max']).float()), Al_AOI)
image_GSAVImax = maskOutside(ee.Image(maxGSAVI25_75perc.select(['GSAVI_max']).float()), Al_AOI)
image_MSAVImax = maskOutside(ee.Image(maxMSAVI25_75perc.select(['MSAVI_max']).float()), Al_AOI)
image_IPVImax = maskOutside(ee.Image(maxIPVI25_75perc.select(['IPVI_max']).float()), Al_AOI)
image_SARVImax = maskOutside(ee.Image(maxSARVI25_75perc.select(['SARVI_max']).float()), Al_AOI)
image_GRVImax = maskOutside(ee.Image(maxGRVI25_75perc.select(['GRVI_max']).float()), Al_AOI)



In [ ]:
# Convert the reduced image of spectral Bands IQR to float32
image_B4mean = maskOutside(ee.Image(compositeVIspectralBands.select(['B4_mean']).float()), Al_AOI)
image_B3mean = maskOutside(ee.Image(compositeVIspectralBands.select(['B3_mean']).float()), Al_AOI)
image_B2mean = maskOutside(ee.Image(compositeVIspectralBands.select(['B2_mean']).float()), Al_AOI)
image_B8mean = maskOutside(ee.Image(compositeVIspectralBands.select(['B8_mean']).float()), Al_AOI)
image_B4min = maskOutside(ee.Image(compositeVIspectralBands.select(['B4_min']).float()), Al_AOI)
image_B3min = maskOutside(ee.Image(compositeVIspectralBands.select(['B3_min']).float()), Al_AOI)
image_B2min = maskOutside(ee.Image(compositeVIspectralBands.select(['B2_min']).float()), Al_AOI)
image_B8min = maskOutside(ee.Image(compositeVIspectralBands.select(['B8_min']).float()), Al_AOI)
image_B4max = maskOutside(ee.Image(compositeVIspectralBands.select(['B4_max']).float()), Al_AOI)
image_B3max = maskOutside(ee.Image(compositeVIspectralBands.select(['B3_max']).float()), Al_AOI)
image_B2max = maskOutside(ee.Image(compositeVIspectralBands.select(['B2_max']).float()), Al_AOI)
image_B8max = maskOutside(ee.Image(compositeVIspectralBands.select(['B8_max']).float()), Al_AOI)
image_B4std = maskOutside(ee.Image(compositeVIspectralBands.select(['B4_stdDev']).float()), Al_AOI)
image_B3std = maskOutside(ee.Image(compositeVIspectralBands.select(['B3_stdDev']).float()), Al_AOI)
image_B2std = maskOutside(ee.Image(compositeVIspectralBands.select(['B2_stdDev']).float()), Al_AOI)
image_B8std = maskOutside(ee.Image(compositeVIspectralBands.select(['B8_stdDev']).float()), Al_AOI)



In [ ]:
image_B4mean.getInfo()

### ***FEATURE SCALING***

Normalisation or Standardization of Vegetation indices

In [ ]:
# Compute standardised median image
# ndvi1999 = NIR.subtract(Red).divide(NIR.add(Red))

# #maskOutside(ee.Image(filtered_grvi2575.qualityMosaic('GRVI').select(['GRVI']).float()), Al_AOI)

# Reduce IRQ of Vegetation index to max value (i.e. the best/highest value)
# Standard max of IQR by subtracting the Mean IQR and dividing by SD of IQR
# standard_NDVI = maskOutside(ee.Image(filtered_ndvi75.qualityMosaic('NDVI').select(['NDVI']).float()), Al_AOI).subtract(image_NDVImean).divide(image_NDVIstD)

standard_NDVI = image_NDVImed.subtract(image_NDVImean).divide(image_NDVIstD)
standard_SAVI = image_SAVImed.subtract(image_SAVImean).divide(image_SAVIstD)
standard_GSAVI = image_GSAVImed.subtract(image_GSAVImean).divide(image_GSAVIstD)
standard_MSAVI = image_MSAVImed.subtract(image_MSAVImean).divide(image_MSAVIstD)
standard_IPVI = image_IPVImed.subtract(image_IPVImean).divide(image_IPVIstD)
standard_SARVI = image_SARVImed.subtract(image_SARVImean).divide(image_SARVIstD)

standard_GRVI = image_GRVImax.subtract(image_GRVImean).divide(image_GRVIstD)


# Visualise computed image
Map.addLayer(standard_GRVI, rgb_vis, "GRVI Standardised IQR")
Map

# Map standardisation across each NDVI image in collection



In [ ]:
spectral_BandsIQR = {"image_B4mean":image_B4mean ,"image_B3mean":image_B3mean,"image_B2mean":image_B2mean,"image_B8mean":image_B8mean, "image_B4min":image_B4min, "image_B3min":image_B3min,
                     "image_B2min":image_B2min,"image_B8min":image_B8min,"image_B4max":image_B4max, "image_B3max":image_B3max, "image_B2max":image_B2max,
                     "image_B8max":image_B8max, "image_B4std":image_B4std, "image_B3std":image_B3std, "image_B2std":image_B2std, "image_B8std":image_B8std}

In [ ]:
spectral_BandsIQR2 = { "image_B4max":image_B4max, "image_B3max":image_B3max, "image_B2max":image_B2max,
                     "image_B8max":image_B8max, "image_B4std":image_B4std, "image_B3std":image_B3std, "image_B2std":image_B2std, "image_B8std":image_B8std}

In [ ]:
for i in range(len(spectral_BandsIQR)):
  name = spectral_BandsIQR[i]
  Bandname = spectral_BandsIQR[i]

  fname = ee.String(Bandname).getInfo()
  #print('2023AlUla_%s_IQR'%(name))
  spectral_BandsIQR[i].getInfo()

In [ ]:
for key, value in spectral_BandsIQR.items():
  fname = ee.String(key).getInfo()
  print('2023AlUla_IQR_'+value)
  #Bandname = spectral_BandsIQR[i]


In [ ]:

# Export the Spectral Band IQR Images
for key, value in spectral_BandsIQR2.items():
  fname = ee.String(key).getInfo()

  task = ee.batch.Export.image.toDrive(
    #image = Bandname.replace('"', ''),
    #image = Bandname.strip(),
    image = value,
    region = Al_AOI.geometry().bounds(), # Or use custom ee.Geometry.Rectangle([minlon, minlat, maxlon, maxlat])
    description = '2023AlUla_IQR_'+fname,
    scale = 10,
    folder = "KSA_AlUla_SprectralBands_IQR",
    crs = 'EPSG:4326',
    #crs ='EPSG:3857', # UTM CRS for Saudi
    #crsTransform = proj["transform"],
    maxPixels = 1e13,
    fileFormat = "GeoTIFF",
    formatOptions = {
    'cloudOptimized': True
  }
    )

  task.start()

  import time

  while task.active():
    counter = 0
    print('Polling for task (id: {}).process: {}'.format(task.id, counter))
    counter += 1
    time.sleep(5)





In [ ]:
# Get location of the band rasters images (.tif files) in Google Drive
VIbands_folder_path = "/content/drive/My Drive/AOI_SprectralBands_IQR/"

# Get the names of all band rasters in the folder as a list

VIband_rasters = [os.path.join(VIbands_folder_path, f) for f in os.listdir(VIbands_folder_path) if f.endswith(".tif")]


In [ ]:
VIband_rasters

In [ ]:
# Open the first band rasteR
# Stacking multiple .tif files into a multiple band stack using Rasterio
# Using Rasterio to stack multiple bands without using subprocess commands
#

import rasterio

#file_list = ['file1.tif', 'file2.tif', 'file3.tif']

# Read metadata of first file
with rasterio.open(VIband_rasters[0]) as src0:
    meta = src0.meta

# Update meta to reflect the number of layers
meta.update(count = len(VIband_rasters))

# Read each layer and write it to stack
with rasterio.open('VIBandsstack.tif', 'w', **meta) as dst:
    for id, layer in enumerate(VIband_rasters, start=1):
        with rasterio.open(layer) as src1:
            dst.write_band(id, src1.read(1))

### Exporting the Standardised Images

In [ ]:
# Export the Standardised IQR Images

task = ee.batch.Export.image.toDrive(
    image = standard_GRVI,
    region = Al_AOI.geometry().bounds(), # Or use custom ee.Geometry.Rectangle([minlon, minlat, maxlon, maxlat])
    description = '2023AlUla_GRVI_stdised_medIQR',
    scale = 10,
    folder = "KSA_AlUla_Standardised_medIQR",
    crs = 'EPSG:4326',
    #crs ='EPSG:3857', # UTM CRS for Saudi
    #crsTransform = proj["transform"],
    maxPixels = 1e13,
    fileFormat = "GeoTIFF",
    formatOptions = {
    'cloudOptimized': True
  }
    )

task.start()

import time
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

In [ ]:
# Get the original CRS and geotransform of the image
#proj = image.projection().getInfo()

# Create a filename for the downloaded image
#filename = image_id.split("/")[-1]

# Export the image with the original CRS and geotransform
task = ee.batch.Export.image.toDrive(
    image = image_GRVImean,
    region = Al_AOI.geometry().bounds(), # Or use custom ee.Geometry.Rectangle([minlon, minlat, maxlon, maxlat])
    description = '2023AlUlaGRVImeanIQR',
    scale = 10,
    folder = "KSA_AlUla_MeanVI_IQR_20230725",
    crs = 'EPSG:4326',
    #crs ='EPSG:3857', # UTM CRS for Saudi
    #crsTransform = proj["transform"],
    maxPixels = 1e13,
    fileFormat = "GeoTIFF",
    formatOptions = {
    'cloudOptimized': True
  }
    )

task.start()

import time
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)
#print(f"Exporting {filename}...")

In [ ]:
# Export the image with the original CRS and geotransform
task = ee.batch.Export.image.toDrive(
    image = image_GRVIstD,
    region = Al_AOI.geometry().bounds(), # Or use custom ee.Geometry.Rectangle([minlon, minlat, maxlon, maxlat])
    description = '2023AlUlaGRVIstdIQR',
    scale = 10,
    folder = "KSA_AlUla_stdDevVI_IQR_202311o2",
    crs = 'EPSG:4326',
    #crs = 'EPSG:3857',
    #crs = proj["crs"],
    #crsTransform = proj["transform"],
    maxPixels = 1e13,
    fileFormat = "GeoTIFF",
    formatOptions = {
    'cloudOptimized': True
  }
    )

task.start()

import time
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)



In [ ]:
# Export the image with the original CRS and geotransform
task = ee.batch.Export.image.toDrive(
    image = image_VImean,
    region = Al_AOI.geometry().bounds(), # Or use custom ee.Geometry.Rectangle([minlon, minlat, maxlon, maxlat])
    description = '2023AlUlaVI_IQRmean',
    scale = 10,
    folder = "KSA_AlUla_MeanVI_IQR_20230725b",
    crs = 'EPSG:3857',
    #crs = proj["crs"],
    #crsTransform = proj["transform"],
    maxPixels = 1e13,
    fileFormat = "GeoTIFF",
    formatOptions = {
    'cloudOptimized': True
  }
    )

task.start()

import time
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)


### Exporting each band as one image

In [ ]:
# To see what you are doing before starting the download loop, you could generate a plot using e.g.

import geemap
Map = geemap.Map()

center = Al_AOI.geometry().centroid().getInfo()

Map.setCenter(center["coordinates"][0], center["coordinates"][1], 6)

Map.addLayer(compositeMeanVIndices.clip(Al_AOI), {}, "Layer")

Map.addLayer(Al_AOI.geometry().bounds(), {}, 'Rectangle')
Map


### GET THE DEM (Elevation) Data for Analysis:Thresholding

The DEM is from the USGS SRTMGL1 30m resolution Data

In [ ]:
# Print the elevation of Mount Everest.
demModelglobal = ee.Image('USGS/SRTMGL1_003')

#xy = ee.Geometry.Point([86.9250, 27.9881])

#elev = dem.sample(xy, 30).first().get('elevation').getInfo()

#print('Mount Everest elevation (m):', elev)

In [ ]:
# Function to mask all values outside the geometry
def maskOutside(image, geometry):
    mask = ee.Image.constant(1).clip(geometry).mask() # add .not() to mask inside
    return image.updateMask(mask)

# Apply the maskOutside() function to clip DEM image and Replace all values outside region with -9999

#FFire_DEM = (maskOutside(demModelglobal, KSA_ForestFire_Area)).float()
#FFire_DEM = maskOutside(ee.Image(demModelglobal).float(), KSA_ForestFire_Area)

#AlUlaProjec_DEM = maskOutside(ee.Image(demModelglobal).float(), Al_AOI)

Al_AOI_DEM = maskOutside(ee.Image(demModelglobal).float(), Al_AOI)
